In [1]:
import pandas as pd
from datetime import datetime
import json

In [2]:
df = pd.read_csv("explored_spotify_data.csv")

In [3]:
df.head()

,ts,platform,ms_played,conn_country,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,reason_start,reason_end,...,offline_timestamp,incognito_mode,hour,day,month,minutes_played,track_artist,time_of_day,album_type_x,album_type_y
0,2025-01-01 05:11:45+00:00,osx,158443,TT,deja vu,Olivia Rodrigo,SOUR,spotify:track:6HU7h9RYOaPRFeh0R3UeAr,trackdone,endplay,...,1735679560,False,5,Wednesday,2025-01,2.640717,deja vu - Olivia Rodrigo,Morning,Album,Album
1,2025-01-01 05:14:52+00:00,osx,188306,TT,Crash My Car,COIN,Dreamland,spotify:track:5SN3mwuodiwY3jPejBuUD5,clickrow,trackdone,...,1735708304,False,5,Wednesday,2025-01,3.138433,Crash My Car - COIN,Morning,Single,Single
2,2025-01-01 05:18:50+00:00,osx,177280,TT,Everybody Talks,Neon Trees,Picture Show,spotify:track:2iUmqdfGZcHIhS3b9E9EWq,trackdone,trackdone,...,1735708522,False,5,Wednesday,2025-01,2.954667,Everybody Talks - Neon Trees,Morning,Single,Single
3,2025-01-01 05:22:13+00:00,osx,202496,TT,She Looks So Perfect,5 Seconds of Summer,5 Seconds Of Summer,spotify:track:1CQ2cMfrmFM1YdfmjENKVE,trackdone,trackdone,...,1735708730,False,5,Wednesday,2025-01,3.374933,She Looks So Perfect - 5 Seconds of Summer,Morning,Single,Single
4,2025-01-01 05:25:51+00:00,osx,218013,TT,Tongue Tied,GROUPLOVE,Never Trust a Happy Song,spotify:track:0GO8y8jQk1PkHzS31d699N,trackdone,trackdone,...,1735708933,False,5,Wednesday,2025-01,3.633550,Tongue Tied - GROUPLOVE,Morning,Single,Single


Now that we’ve explored and visualized the data, it’s time to transform the data so future insights (and predictions, if any!) are richer.

**Feature engineering** is where you make the raw data work for you.
At first, I thought that it was a scary and complex term but it's just preparing your data for predictions.

There are five features we can engineer here (haha).
1. Time-based features - understanding when I listen and how often
2. User Behaviour features - quantifying how I listen
3. Song-level features - measuring popularity or repetition
4. Artist features - who do I love?
5. Categorical encoding (for more modeling) - one-hot encoding

#### Time-Based Features
Understanding when and how often I listen to music.
- **hour_of_day**: I typically listen to music in the late night (hr: 01, 02, 00, 23, 03)
- **day_of_week**: I have the highest listening activity on Thursdays and the lowest on Fridays. The each day of the week has relatively similar listening times, though.
- **month**: First, recall that this project takes data from January 1st to mid-July. June had the greatest listening time, and January, the least.

#### User-Behaviour Features
Quantifying how I listen to music.
- **session_id**: grouping plays into listening sessions (e.g. gaps >30 minutes = new session)
- **songs_per_session**: how many tracks per session.
- **minutes_per_session**: total lsitening time per session.

In [9]:
df['ts'] = pd.to_datetime(df['ts'], errors='coerce')

In [11]:
df['time_diff'] = df['ts'].diff().dt.total_seconds().fillna(0)
df['new_session'] = df['time_diff'] > (30 * 60)
df['session_id'] = df['new_session'].cumsum()

df.tail()

,ts,platform,ms_played,conn_country,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,reason_start,reason_end,...,day,month,minutes_played,track_artist,time_of_day,album_type_x,album_type_y,time_diff,new_session,session_id
10915,2025-07-17 19:15:27+00:00,ios,171789,TT,RUSH,Dutch Melrose,RUSH,spotify:track:7Ey8DZuMBbeiqMk8s8XpHA,fwdbtn,trackdone,...,Thursday,2025-07,2.863150,RUSH - Dutch Melrose,Evening,Single,Single,197.0,False,699
10916,2025-07-17 19:21:40+00:00,ios,177729,TT,numb (feat. blackbear),lilspirit,numb (feat. blackbear),spotify:track:05UKPuq9D7C6YDaps38ReU,trackdone,trackdone,...,Thursday,2025-07,2.962150,numb (feat. blackbear) - lilspirit,Evening,Single,Single,373.0,False,699
10917,2025-07-17 19:25:44+00:00,ios,173866,TT,Sunscreen,Ax and the Hatchetmen,Sunscreen,spotify:track:7ndvAM7pJXTUOJmvypg1HX,trackdone,trackdone,...,Thursday,2025-07,2.897767,Sunscreen - Ax and the Hatchetmen,Evening,Single,Single,244.0,False,699
10918,2025-07-17 19:32:14+00:00,ios,145454,TT,Bleed,Connor Kauffman,Bleed,spotify:track:4k8D363TGNjILbQRciQfBD,trackdone,trackdone,...,Thursday,2025-07,2.424233,Bleed - Connor Kauffman,Evening,Single,Single,390.0,False,699
10919,2025-07-17 19:34:59+00:00,ios,165392,TT,runaway bride,Ellise,PRETTY EVIL DELUXE,spotify:track:0ueaaxNQtKa5m3TbTXLyIH,trackdone,trackdone,...,Thursday,2025-07,2.756533,runaway bride - Ellise,Evening,Album,Album,165.0,False,699


In [13]:
songs_per_session = df.groupby('session_id')['master_metadata_track_name'].count()
songs_per_session

session_id
0       7
1       1
2       1
3       8
4       5
       ..
695     8
696    26
697    14
698     7
699     7
Name: master_metadata_track_name, Length: 700, dtype: int64

In [14]:
df['duration_minutes'] = df['ms_played'] / (1000 * 60)
minutes_per_session = df.groupby('session_id')['duration_minutes'].sum()
minutes_per_session

session_id
0      22.145400
1       1.118417
2       2.543250
3      28.535333
4      13.081650
         ...    
695    24.390983
696    84.550533
697    38.901567
698    17.435117
699    17.907383
Name: duration_minutes, Length: 700, dtype: float64

#### Song-Level Features
Measuring popularity or repetition
- **total_plays**
- **total_minutes**
- **avg_duration**: total_minutes / total_plays

In [17]:
# Total plays and minutes played per song
song_stats = df.groupby('track_artist').agg(
    total_plays = ('master_metadata_track_name', 'count'),
    total_minutes = ('duration_minutes', 'sum')
)

song_stats['avg_duration'] = song_stats['total_minutes'] / song_stats['total_plays']
song_stats

,total_plays,total_minutes,avg_duration
track_artist,,,
"""Honest"" - Nico Collins",9,28.792050,3.199117
(Baby) Come On Back - THE BAND LIGHT,1,2.662567,2.662567
07 Britney - Acoustic - Chelsea Collins,1,3.079400,3.079400
07 Britney - Chelsea Collins,5,15.242917,3.048583
"1 step forward, 3 steps back - Olivia Rodrigo",1,2.711467,2.711467
...,...,...,...
夜に駆ける - YOASOBI,2,8.634833,4.317417
愛して 愛して 愛して - Kikuo,2,8.400000,4.200000
残響散歌 - Aimer,1,3.081550,3.081550


#### Artist Features
Who do I love?
- **most_played_artist**: which, as of this project, is Ellise.
- **artist_diversity**: number of unique artist / total plays.

In [18]:
# Most Played Artist
artist_stats = df.groupby('master_metadata_album_artist_name').agg(
    total_plays = ('master_metadata_album_artist_name', 'count'),
    total_minutes = ('duration_minutes', 'sum')
).sort_values('total_plays', ascending=False)

artist_stats

,total_plays,total_minutes
master_metadata_album_artist_name,,
Ellise,1134,3100.794050
Vlad Holiday,784,2609.252283
Nico & Chelsea,502,2050.642000
Waterparks,500,1477.019333
WesGhost,478,1298.274417
...,...,...
The Grogans,1,2.112150
The Brummies,1,3.411333
That Mexican OT,1,0.512383


In [22]:
# Artist Diversity Score
unique_artists = df['master_metadata_album_artist_name'].nunique()
total_plays = len(df)
artist_diversity = (unique_artists / total_plays) * 100

print(f'Artist Diversity = ', artist_diversity, '%')

Artist Diversity =  4.697802197802198 %


#### Categorical Encoding
- **day**: day of the week.
- **hour**: hour of the day binned into categories.

These are already in our dataframe!

In [23]:
df.columns

Index(['ts', 'platform', 'ms_played', 'conn_country',
       'master_metadata_track_name', 'master_metadata_album_artist_name',
       'master_metadata_album_album_name', 'spotify_track_uri', 'reason_start',
       'reason_end', 'shuffle', 'skipped', 'offline', 'offline_timestamp',
       'incognito_mode', 'hour', 'day', 'month', 'minutes_played',
       'track_artist', 'time_of_day', 'album_type_x', 'album_type_y',
       'time_diff', 'new_session', 'session_id', 'duration_minutes'],
      dtype='object')

#### Bonus Features
For funsies?

**Top N Flags**: boolean columns like *is_top_artist* and *is_top_song* check whether the artist or song is in the top N (let's use 10).

In [25]:
# Get top 10 artists and songs
top_artists = df['master_metadata_album_artist_name'].value_counts().head(10).index
top_tracks = df['master_metadata_track_name'].value_counts().head(10).index

# Create boolean columns
df['is_top_artist'] = df['master_metadata_album_artist_name'].isin(top_artists)
df['is_top_track'] = df['master_metadata_track_name'].isin(top_tracks)


In [27]:
df.tail()

,ts,platform,ms_played,conn_country,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,reason_start,reason_end,...,track_artist,time_of_day,album_type_x,album_type_y,time_diff,new_session,session_id,duration_minutes,is_top_artist,is_top_track
10915,2025-07-17 19:15:27+00:00,ios,171789,TT,RUSH,Dutch Melrose,RUSH,spotify:track:7Ey8DZuMBbeiqMk8s8XpHA,fwdbtn,trackdone,...,RUSH - Dutch Melrose,Evening,Single,Single,197.0,False,699,2.863150,False,False
10916,2025-07-17 19:21:40+00:00,ios,177729,TT,numb (feat. blackbear),lilspirit,numb (feat. blackbear),spotify:track:05UKPuq9D7C6YDaps38ReU,trackdone,trackdone,...,numb (feat. blackbear) - lilspirit,Evening,Single,Single,373.0,False,699,2.962150,False,False
10917,2025-07-17 19:25:44+00:00,ios,173866,TT,Sunscreen,Ax and the Hatchetmen,Sunscreen,spotify:track:7ndvAM7pJXTUOJmvypg1HX,trackdone,trackdone,...,Sunscreen - Ax and the Hatchetmen,Evening,Single,Single,244.0,False,699,2.897767,False,False
10918,2025-07-17 19:32:14+00:00,ios,145454,TT,Bleed,Connor Kauffman,Bleed,spotify:track:4k8D363TGNjILbQRciQfBD,trackdone,trackdone,...,Bleed - Connor Kauffman,Evening,Single,Single,390.0,False,699,2.424233,False,False
10919,2025-07-17 19:34:59+00:00,ios,165392,TT,runaway bride,Ellise,PRETTY EVIL DELUXE,spotify:track:0ueaaxNQtKa5m3TbTXLyIH,trackdone,trackdone,...,runaway bride - Ellise,Evening,Album,Album,165.0,False,699,2.756533,True,False


**Daily Listen Counts**: counts how many listens per day.

In [29]:
# Daily Listen Count
df['date'] = df['ts'].dt.date
daily_counts = df.groupby('date').size().reset_index(name='daily_listen_count')
df = df.merge(daily_counts, on='date')

df.head()

,ts,platform,ms_played,conn_country,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,reason_start,reason_end,...,album_type_y,time_diff,new_session,session_id,duration_minutes,is_top_artist,is_top_track,date,daily_listen_count_x,daily_listen_count_y
0,2025-01-01 05:11:45+00:00,osx,158443,TT,deja vu,Olivia Rodrigo,SOUR,spotify:track:6HU7h9RYOaPRFeh0R3UeAr,trackdone,endplay,...,Album,0.0,False,0,2.640717,False,False,2025-01-01,22,22
1,2025-01-01 05:14:52+00:00,osx,188306,TT,Crash My Car,COIN,Dreamland,spotify:track:5SN3mwuodiwY3jPejBuUD5,clickrow,trackdone,...,Single,187.0,False,0,3.138433,False,False,2025-01-01,22,22
2,2025-01-01 05:18:50+00:00,osx,177280,TT,Everybody Talks,Neon Trees,Picture Show,spotify:track:2iUmqdfGZcHIhS3b9E9EWq,trackdone,trackdone,...,Single,238.0,False,0,2.954667,False,False,2025-01-01,22,22
3,2025-01-01 05:22:13+00:00,osx,202496,TT,She Looks So Perfect,5 Seconds of Summer,5 Seconds Of Summer,spotify:track:1CQ2cMfrmFM1YdfmjENKVE,trackdone,trackdone,...,Single,203.0,False,0,3.374933,False,False,2025-01-01,22,22
4,2025-01-01 05:25:51+00:00,osx,218013,TT,Tongue Tied,GROUPLOVE,Never Trust a Happy Song,spotify:track:0GO8y8jQk1PkHzS31d699N,trackdone,trackdone,...,Single,218.0,False,0,3.633550,False,False,2025-01-01,22,22


**Favourite Hour Bucket**: creates a *fave_hour* feature that shows the hour of the day I most frequently listen to music.

In [30]:
fave_hour = df['hour'].value_counts().idxmax()
df['is_favourite_hour'] = df['hour'] == fave_hour

df.head()

,ts,platform,ms_played,conn_country,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,reason_start,reason_end,...,time_diff,new_session,session_id,duration_minutes,is_top_artist,is_top_track,date,daily_listen_count_x,daily_listen_count_y,is_favourite_hour
0,2025-01-01 05:11:45+00:00,osx,158443,TT,deja vu,Olivia Rodrigo,SOUR,spotify:track:6HU7h9RYOaPRFeh0R3UeAr,trackdone,endplay,...,0.0,False,0,2.640717,False,False,2025-01-01,22,22,False
1,2025-01-01 05:14:52+00:00,osx,188306,TT,Crash My Car,COIN,Dreamland,spotify:track:5SN3mwuodiwY3jPejBuUD5,clickrow,trackdone,...,187.0,False,0,3.138433,False,False,2025-01-01,22,22,False
2,2025-01-01 05:18:50+00:00,osx,177280,TT,Everybody Talks,Neon Trees,Picture Show,spotify:track:2iUmqdfGZcHIhS3b9E9EWq,trackdone,trackdone,...,238.0,False,0,2.954667,False,False,2025-01-01,22,22,False
3,2025-01-01 05:22:13+00:00,osx,202496,TT,She Looks So Perfect,5 Seconds of Summer,5 Seconds Of Summer,spotify:track:1CQ2cMfrmFM1YdfmjENKVE,trackdone,trackdone,...,203.0,False,0,3.374933,False,False,2025-01-01,22,22,False
4,2025-01-01 05:25:51+00:00,osx,218013,TT,Tongue Tied,GROUPLOVE,Never Trust a Happy Song,spotify:track:0GO8y8jQk1PkHzS31d699N,trackdone,trackdone,...,218.0,False,0,3.633550,False,False,2025-01-01,22,22,False


In [31]:
fave_hour

np.int64(1)

**Listening Format Estimation (or Album-Style Listening)**: approximates when I'm listening to a full album straight through.

In [37]:
# First and foremost, data is already sorted by timestamp

# Check how often tracks from the same album are played consecutively
df['same_album_as_previous'] = (df['master_metadata_album_album_name'] == df['master_metadata_album_album_name'].shift(1)) & (df['ts'] - df['ts'].shift(1) < pd.Timedelta(minutes=10))

# Rolling album sequence
df['album_listen_block'] = df['same_album_as_previous'].cumsum()

**Listen Streak**: how many days in a row do I listen to music.

In [40]:
df['ts'] = pd.to_datetime(df['ts'])
df['date'] = df['ts'].dt.date

# Drop duplicates to avoid counting multiple listens per day
unique_dates = df[['date']].drop_duplicates().sort_values('date')

# Calculate streak
unique_dates['prev_date'] = unique_dates['date'].shift(1)
unique_dates['gap'] = unique_dates['date'] - unique_dates['prev_date']
unique_dates['new_streak'] = unique_dates['gap'].apply(lambda x: x.days > 1 if pd.notnull(x) else True)
unique_dates['streak_id'] = unique_dates['new_streak'].cumsum()

# Get streak lengths
streak_lengths = unique_dates.groupby('streak_id').size().reset_index(name='length')

# Longest streak
longest_streak = streak_lengths['length'].max()
print(f"Longest Listening Streak: {longest_streak} days")

Longest Listening Streak: 171 days


In [41]:
days_in_project = 31 + 28 + 31 + 30 + 31 + 30 + 17
print(f"The project contains data over {days_in_project} days")

The project contains data over 198 days


In [42]:
streak_score = (longest_streak / days_in_project) * 100
print(f"The longest listening streak is {streak_score}% of the length of this project")

The longest listening streak is 86.36363636363636% of the length of this project


In [43]:
# Save as csv file to load in another notebook
df.to_csv("engineered_spotify_data.csv", index=False)